In [1]:
import numpy as np
import mindspore
import mindspore.dataset as ds
import mindspore.ops as ops
from mindspore import Tensor
from d2l import mindspore as d2l

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [2]:
class SyntheticData():
    def __init__(self):
        self.features, self.labels = d2l.synthetic_data(true_w, true_b, 1000)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    
    def __len__(self):
        return len(self.labels)

def load_array(data_arrays, column_names, batch_size, is_train=True):  
    """构造一个MindSpore数据迭代器。"""
    dataset = ds.GeneratorDataset(data_arrays, column_names, shuffle=is_train)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = SyntheticData()
dataset = load_array(data_iter, ['features', 'labels'], batch_size)

In [3]:
next(iter(dataset))

[Tensor(shape=[10, 2], dtype=Float32, value=
 [[ 1.73414111e-01, -1.24616718e+00],
  [ 1.02213132e+00, -6.50639892e-01],
  [ 1.30825520e+00, -1.22844315e+00],
  ...
  [-1.60054636e+00,  1.20353259e-01],
  [ 7.61325806e-02,  7.83914328e-01],
  [ 1.73925209e+00, -3.65485922e-02]]),
 Tensor(shape=[10, 1], dtype=Float32, value=
 [[ 8.76539135e+00],
  [ 8.46442604e+00],
  [ 1.09844027e+01],
  ...
  [ 6.07668221e-01],
  [ 1.68395686e+00],
  [ 7.80520296e+00]])]

In [4]:
# nn是神经网络的缩写
from mindspore import nn
from mindspore.common.initializer import initializer, Normal

net = nn.SequentialCell([nn.Dense(2, 1)])

In [5]:
net[0].weight = initializer(Normal(), net[0].weight.shape, mindspore.float32)
net[0].bias = initializer('zero', net[0].bias.shape, mindspore.float32)

In [6]:
loss = nn.MSELoss()

In [7]:
optimizer = nn.SGD(net.trainable_params(), learning_rate=0.03)

In [8]:
# 构造前向网络
def forward_fn(x, y):
    y_hat = net(x)
    l = loss(y_hat, y)
    return l
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in dataset:
        grad_fn = mindspore.value_and_grad(forward_fn, grad_position=None, weights=optimizer.parameters)
        l, grads = grad_fn(X, y)
        optimizer(grads)
    l = forward_fn(mindspore.Tensor(data_iter.features), mindspore.Tensor(data_iter.labels))
    print(f'epoch {epoch + 1}, loss {l.asnumpy():f}')

epoch 1, loss 0.000214
epoch 2, loss 0.000108
epoch 3, loss 0.000108


In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： [Tensor(shape=[], dtype=Float32, value= -0.00017786)
 Tensor(shape=[], dtype=Float32, value= -0.000656366)]
b的估计误差： [-0.00027466]
